In [1]:
from bids import BIDSLayout, BIDSLayoutIndexer
import pandas as pd
import numpy as np
bids_path = "/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1/"
motion_threshold = 0.5

# get subjects from bids 
indexer = BIDSLayoutIndexer(validate=False, index_metadata=False)
layout = BIDSLayout(bids_path, indexer=indexer)
subject_ids = layout.get_subjects()
subject_ids.remove("159")  # no clinical data (yet)

for sid in subject_ids:
    files = layout.get(subject=sid, extension=".tsv", return_type="filenames")
    df = pd.read_csv(files[0], delimiter="\t")
    df2 = pd.read_csv(files[1], delimiter="\t")
    motion = np.hstack((df.framewise_displacement.values[1::], df.framewise_displacement.values[1::]))
    motion_avg = np.mean(motion)

    if motion_avg > motion_threshold:
        print("removing", sid)
        subject_ids.remove(sid)


removing 093


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import nibabel as nb
from nilearn.image import math_img
from nilearn.maskers import NiftiMasker

bids_path = "/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/bids/"
coord_df = pd.read_csv(bids_path+'phenotype/coordinates.tsv',
                        delimiter="\t")


path_to_data = "/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1_denoised_smoothed/"
out_path = "/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1_denoised_smoothed_ts/"

# create mask
gm_img = nb.load("../../QNC_pipeline/masks/GMmask_new_trim_YT-NC.nii")
dlpfc_img = nb.load("../../QNC_pipeline/masks/DLPFCcombo20mm-NC.nii")

# Process normative data by masking out the dlpfc
dlpfc_neg_mask = math_img(
    "-img + 1",
    img=dlpfc_img,
)
gm_img_masked = math_img(
    "img1 * img2",
    img1=gm_img,
    img2=dlpfc_neg_mask,
)

for subj in tqdm(subject_ids):

    for ses in ["pre", "post"]:
        # get 4d nifti data
        fname = f"{path_to_data}sub-{subj}/ses-{ses}/sub-{subj}_ses-{ses}_denoise-Ciric36p_filter-butterworth_0.01_0.10_smooth-4_bold.nii.gz"

        # mask the data
        mask = NiftiMasker(mask_img=gm_img_masked)
        ts = mask.fit_transform(fname)

        out = f"{out_path}sub-{subj}/ses-{ses}/sub-{subj}_ses-{ses}_denoise-Ciric36p_filter-butterworth_0.01_0.10_smooth-4_bold_global.csv"
        np.savetxt(out, ts, delimiter=",")

100%|██████████| 27/27 [3:21:03<00:00, 446.78s/it]  
